In [1]:
import pandas as pd
import numpy as np

In [2]:
ranges = {'Open': (-0.20389109145138629, 0.106712091615353),
 'High': (-0.13116383242318375, 0.08037938471723338),
 'Low': (-0.2013710898518115, 0.0941051203052028),
 'Close': (-0.20466930860972155, 0.11580036960722695)}

def reverse_normalisation(a, c):
    r = np.array([(((v+1.0)/2.0)*(ranges[c][1]-ranges[c][0]))+ranges[c][0] for v in a])
    return(r)

In [3]:
base_columns = ["Open", "High", "Low", "Close"]
index_columns = ["OpenSign", "HighSign", "LowSign", "CloseSign"]
preds = pd.read_csv("../outputs/predictions/sequifier-l3-d48-8targets-best-50-predictions.csv")


In [4]:
for base_column in base_columns:
    preds[base_column] = reverse_normalisation(preds[base_column].values, base_column)

,sequenceId,Open,High,Low,Close,OpenSign,HighSign,LowSign,CloseSign
0,0.0,0.573313,0.210987,0.370813,0.222462,1,1,1,2
1,0.0,0.474971,0.159107,0.411732,0.186980,2,2,2,2
2,0.0,0.486207,0.143382,0.354275,0.206475,2,2,2,2
3,0.0,0.464732,0.210663,0.383934,0.234997,2,2,2,2
4,0.0,0.487568,0.175746,0.369716,0.217762,2,2,2,2
...,...,...,...,...,...,...,...,...,...
2100,0.0,0.436363,0.208990,0.403344,0.207528,2,2,2,2
2101,0.0,0.436364,0.208990,0.403344,0.207528,2,2,2,2
2102,0.0,0.436362,0.208991,0.403344,0.207528,2,2,2,2
2103,0.0,0.436364,0.208989,0.403344,0.207529,2,2,2,2


In [34]:
row_data = pd.read_parquet("../data/gspc-diff-split2.parquet")
targets = {}
for c, group in row_data.groupby("inputCol"):
    targets[c] = reverse_normalisation(group["target"].values, c)


In [45]:
for c in ["Open", "High", "Low", "Close"]:
    print(f"{c}: {np.corrcoef(preds[c], targets[c])[0, 1]}")

Open: 0.6453685067658671
High: 0.31313387977256985
Low: 0.25028372296860496
Close: 0.01532822879606665


In [47]:
dummy_preds = {} # data from prior day
for c, group in row_data.groupby("inputCol"):
    dummy_preds[c] = reverse_normalisation(group["target"].values[:-1], c)
for c in ["Open", "High", "Low", "Close"]:
    print(f"{c}: {np.corrcoef(dummy_preds[c], targets[c][1:])[0, 1]}")

Open: 0.052477568267726385
High: 0.10139333619511509
Low: 0.13659016605936497
Close: -0.09437963121240334


In [41]:
pd.DataFrame(preds)[cols]

,Open,High,Low,Close
0,3.656177,-0.881940,10.987387,2.331830
1,12.056953,3.069358,7.921552,-0.336649
2,7.170805,4.416052,2.095096,-2.842158
3,-22.560573,-9.059299,-10.784293,1.769481
4,7.644910,-0.825959,5.550206,-2.643871
...,...,...,...,...
2724,-23.173972,-9.553667,0.325382,3.746359
2725,11.136063,-6.440793,9.662473,0.364475
2726,19.425856,8.577875,9.938933,-6.977384
2727,12.005382,-3.608873,7.865488,-7.217466


In [42]:
pd.DataFrame(targets)[cols]

,Open,High,Low,Close
0,6.030029,8.559936,16.339966,16.680054
1,8.700074,2.420044,12.000000,13.609985
2,17.789917,-1.540039,2.850098,-23.839965
3,-23.369996,-1.550049,-15.890015,9.630005
4,4.880005,-5.699951,-6.380005,-12.310059
...,...,...,...,...
2724,-52.419922,2.610352,-40.140137,1.949707
2725,-5.659668,-5.240235,36.570313,-7.520019
2726,49.149902,46.379883,22.079590,49.270019
2727,-5.070312,-33.349609,-0.069825,-38.419922
